In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

KeyboardInterrupt: 

In [3]:
df_full_preprocessed = pd.read_pickle("./df_full_preprocessed.pkl")

In [4]:
# use the  text transformer class to create two transformers for the textual and the numerical model
text_transformer = TfidfVectorizer(ngram_range=(1,1), min_df=int(len(df_full_preprocessed)**(1/2)), use_idf=True)
numerical_transformer = make_column_transformer((OneHotEncoder(handle_unknown="ignore"), ["gender","topic","sign"])\
                                                           , remainder=StandardScaler())


In [5]:
# create dataset
X_text = text_transformer.fit_transform(df_full_preprocessed["text_preprocessed"])
X_numerical = numerical_transformer.fit_transform(df_full_preprocessed.drop("text_preprocessed", axis=1))

In [6]:
validation_results = pd.DataFrame(data={'Features': [], 'Sum of squared distances': []})

In [ ]:
# elbow plot for X_text

Sum_of_squared_distances = {}
K = range(2,40)
for k in tqdm(K):
    km = KMeans(n_clusters=k, max_iter=200, random_state=42)
    km = km.fit(X_text)
    Sum_of_squared_distances[k] = km.inertia_
        
new_row = {'Features':'Text', 'Sum of squared distances': Sum_of_squared_distances}
validation_results = validation_results.append(new_row, ignore_index=True)


  0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
# elbow plot for X_numerical

Sum_of_squared_distances = {}
K = range(2,40)
for k in tqdm(K):
    km = KMeans(n_clusters=k, max_iter=200)
    km = km.fit(numerical_features)
    Sum_of_squared_distances[k] = km.inertia_
        
new_row = {'Features':'Numerical', 'Sum of squared distances':Sum_of_squared_distances}
validation_results = validation_results.append(new_row, ignore_index=True)